In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
from deepsith import DeepSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os 
from os.path import join
import glob

import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

In [ ]:
MORSE_CODE_DICT = { 'A':'.-', 'B':'-...', 
                    'C':'-.-.', 'D':'-..', 'E':'.', 
                    'F':'..-.', 'G':'--.', 'H':'....', 
                    'I':'..', 'J':'.---', 'K':'-.-', 
                    'L':'.-..', 'M':'--', 'N':'-.', 
                    'O':'---', 'P':'.--.', 'Q':'--.-', 
                    'R':'.-.', 'S':'...', 'T':'-', 
                    'U':'..-', 'V':'...-', 'W':'.--', 
                    'X':'-..-', 'Y':'-.--', 'Z':'--..', 
                    '1':'.----', '2':'..---', '3':'...--', 
                    '4':'....-', '5':'.....', '6':'-....', 
                    '7':'--...', '8':'---..', '9':'----.', 
                    '0':'-----', ', ':'--..--', '.':'.-.-.-', 
                    '?':'..--..', '/':'-..-.', '-':'-....-', 
                    '(':'-.--.', ')':'-.--.-'} 

In [ ]:
print(MORSE_CODE_DICT['?'], MORSE_CODE_DICT['?'].replace('.', '10').replace('-', '1110'))
print(len(MORSE_CODE_DICT))
morse_code_numpy = {key:np.array([int(x) for x in MORSE_CODE_DICT[key].replace('.', '10').replace('-', '1110')] + [0, 0])
                    for key in MORSE_CODE_DICT.keys()}
for k in morse_code_numpy.keys():
    #if len(morse_code_numpy[k]) == 12:
    #    print(morse_code_numpy[k], k)
    print(morse_code_numpy[k], k)
print(len(morse_code_numpy))
subset = list(morse_code_numpy.keys())
#subset = ['3', '7', 'Y', 'Q', 'J',
#          'M', 'R', 'U', 'H', 'D']

In [ ]:
id2key = subset
key2id = {}
for idx, s in enumerate(subset):
    key2id[s] = idx

X = [ttype(morse_code_numpy[k])for k in subset]
Y = torch.LongTensor(np.arange(0,len(X)))
print(Y.max())
print(X, Y)

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])

# 8 Layers

In [ ]:
model = TCN(1, 43, [25]*8, kernel_size=15, dropout=0.0).cuda()
tot_weights = 0
for p in model.parameters():
    tot_weights += p.numel()
print("Total Weights:", tot_weights)
print(model)


In [ ]:
runs = 5
epochs = 1000
Trainscale = 10
device='cuda'
batch_size = 8
batches = int(np.ceil(43 / batch_size))
for r in range(runs):
    model = TCN(1, 43, [25,25,25,25, 
                        25,25,25,25
                       ], 
                kernel_size=15, dropout=0.0).cuda()
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    perf_file = join('perf','TCN_morsedecoder_run_{}.csv'.format(r))
    
    

    progress_bar = tqdm(range(int(epochs)), bar_format='{l_bar}{bar:5}{r_bar}{bar:-5b}')
    times_100 = 0

    for epoch_idx in progress_bar:
        perfs = []
        losses = []
        model.train()
        for batch_idx in range(batches):
            optimizer.zero_grad()
            loss = 0
            permute = np.arange(0, 43)
            np.random.shuffle(permute)

            for i in range(0, int(min(len(X) - (batch_idx*batch_size), 
                                  batch_size))
                           ):
                iv = X[permute[batch_idx*batch_size + i]]
                iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
                iv = iv.repeat(1,1,1,Trainscale)
                iv = iv.reshape(1,1,-1)
                tv = Y[permute[batch_idx*batch_size + i]].to(device)
                out = model(iv)
                loss += loss_func(out,
                             torch.cuda.LongTensor([tv]))
                perfs.append((torch.argmax(out, dim=-1) == 
                              tv).sum().item())


            loss = loss / min(len(X) - (batch_idx*batch_size), 
                              batch_size)
            loss.backward()
            optimizer.step()


            #perfs = perfs[int(-loss_buffer_size/batch_size):]
            losses.append(loss.detach().cpu().numpy())
            #losses = losses[int(-loss_buffer_size/batch_size):]


            s = "{}:{:2} Loss: {:.4f}, Perf: {:.4f}"
            format_list = [epoch_idx, batch_idx, np.mean(losses), 
                           np.sum(perfs)/((len(perfs)))]
            s = s.format(*format_list)
            progress_bar.set_description(s)
        if (np.sum(perfs)/((len(perfs))) == 1.0) & (np.mean(losses) < .11):
            times_100 += 1
            if times_100 >= 3:
                break
    torch.save(model.state_dict(), perf_file[:-4]+".pt")

In [ ]:
runs = 5
device = 'cuda'
for r in range(runs):
    model.load_state_dict(torch.load(join('perf','TCN_morsedecoder_trun_{}.pt'.format(r))))
    model.eval()
    with torch.no_grad():
        evald = []
        evaldDict = {'test_perf':[],
                     'rate':[]}
        for nr in [1,2,4,8,10,12,25,50,100,125,250,500,1000]:
        #for nr in range(1,20):
            perfs = []
            for batch_idx, iv in enumerate(X):
                iv = iv.unsqueeze(0).unsqueeze(0).unsqueeze(-1)
                iv = iv.repeat(1,1,1,nr)
                iv = iv.reshape(1,1,-1)
                tv = Y[batch_idx].to(device)
                out = model(iv)

                perfs.append((torch.argmax(out, dim=-1) == 
                              tv).sum().item())
                #print(torch.argmax(out, dim=-1), 
                #              tv)
            evaldDict['test_perf'].append(sum(perfs)/len(perfs))
            evaldDict['rate'].append(nr)
            print(nr, sum(perfs)/len(perfs))
            evald.append({'scale':nr, 
                          'perf':sum(perfs)/len(perfs)})
        scale_perfs = pd.DataFrame(evald)
        scale_perfs.to_pickle(join("perf", "tcn_morse_ttest_{}.dill".format(r)))